In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass, fields, asdict
import gc
from tqdm import tqdm

import torch

from model.model import PageAccModel
from logfile_reader import read_pages, read_optimal_results, Page

In [2]:
BUFFER_SIZE = 128
TRAIN_PART = 0.7
RESULTS_DIR = "results/data/tpcc_128/"

In [3]:
pages = read_pages(RESULTS_DIR + "logfile")
train_size = int(len(pages) * TRAIN_PART)
test_size = len(pages) - train_size
# train_pages = pages[:train_size]
test_pages = pages[train_size:]
# victims = read_optimal_results(RESULTS_DIR + "victims")

In [4]:
class IPageVictimStrategy(ABC):
    @abstractmethod
    def forward(page: Page, buffer: list[Page]):
        pass

In [6]:
def get_metrics(pages: list[Page], evict_strategy: IPageVictimStrategy):
    buffer = [Page(-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0)] * BUFFER_SIZE

    hits = 0
    hit_rates = []
    pbar = tqdm(range(len(pages)))
    for i in pbar:
        page = pages[i]
        victim = evict_strategy.forward(page, buffer)
        if victim < 0:
            hits += 1
        else:
            # buffer[victim] = deepcopy(page)
            buffer[victim] = page
            buffer[victim].hit = victim
        
        hit_rates.append(hits / (i + 1))

        if i % 1000 == 0:
            pbar.set_postfix_str(f"hit_rate={hits / (i + 1)}")
    
    return hit_rates

In [9]:
class ModelVictimStrategy(IPageVictimStrategy):
    def __init__(self, model: PageAccModel):
        self._model = model
        self._h = None
        self._c = None

        self._model.eval()

    def forward(self, page: Page, buffer: list[Page]):
        page_tensor = torch.Tensor(list(asdict(page).values()))
        page_in_buffer = next(filter(lambda x: x[1].get_page_id() == page.get_page_id(), enumerate(buffer)), None)
        page_already_in_buf = page_in_buffer is not None
        if page_already_in_buf:
            page_tensor[-1] = int(page_in_buffer[0])
        page_tensor = page_tensor.view(1, len(page_tensor))

        buffer_tensor = torch.Tensor([value for obj in buffer for value in asdict(obj).values()])
        buffer_tensor = buffer_tensor.view(1, len(buffer_tensor))

        with torch.no_grad():
            out, self._h, self._c = self._model.forward(page_tensor, buffer_tensor, page_already_in_buf, self._h, self._c)

        self._h.detach()
        self._c.detach()

        if page_already_in_buf:
            return -1

        return torch.argmax(out, dim=1)[0]

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PageAccModel(len(fields(Page)), 256, 512, BUFFER_SIZE)
model.load_state_dict(torch.load(RESULTS_DIR + "model.pth", map_location=device, weights_only=True))
model.eval()

model_evict_strategy = ModelVictimStrategy(model)

In [ ]:
hit_rates = get_metrics(test_pages, model_evict_strategy)

  0%|          | 0/663413 [00:00<?, ?it/s]

  1%|          | 4718/663413 [00:46<1:47:02, 102.56it/s, hit_rate=0.23919020244938766]


KeyboardInterrupt: 

In [ ]:
with open("hit_results", "w") as f:
    for hit_rate in hit_rates:
        f.write(f"{hit_rate}\n")
